In [3]:
import ujson
import nltk
import glob
import os
import time
import pandas as pd
import numpy as np

In [7]:
# load the java document caveats
caveat_files_dir = './output/java_12_spec_caveat_sentences_revised/'
caveats_dict = {}
class_to_caveat_ids = {}

files = glob.glob(caveat_files_dir + '*.json')
for file in files:
    with open(file) as f:
        arr = ujson.load(f)
        full_class_name = os.path.splitext(os.path.basename(file))[0]
        simple_class_name = full_class_name.split('.')[-1]
        for caveat in arr:
            caveats_dict[caveat['id']] = caveat
            
            if simple_class_name not in class_to_caveat_ids:
                class_to_caveat_ids[simple_class_name] = []
            class_to_caveat_ids[simple_class_name].append(caveat['id'])

In [47]:
caveat_sentences_array = []
sentence_id_to_caveat_id = {}
   

for key in caveats_dict:
    caveat = caveats_dict[key]
    
    if caveat['deprecated']:
        continue
 
    if 'class_caveat_sentences' in caveat:
        for sentence in caveat['class_caveat_sentences']:
            if len(sentence) > 0:
                sentence_id_to_caveat_id[len(caveat_sentences_array)] = caveat['id']
                caveat_sentences_array.append({
                    'text': sentence, 
                     'id': len(caveat_sentences_array),
                     'type': 'class'
                })
    else:
        for sentence in caveat['sentences']:
            if len(sentence) > 0:
                sentence_id_to_caveat_id[len(caveat_sentences_array)] = caveat['id']
                caveat_sentences_array.append({
                    'text': sentence, 
                     'id': len(caveat_sentences_array),
                     'type': caveat['type']
                })
            # add sentences for throws, returns etc...
            if caveat['type'] in ['method', 'constructor']:
                for misc in caveat['caveat_misc']:
                    for misc_obj in misc['list']:
                        if type(misc_obj) == str:
                            sentence_id_to_caveat_id[len(caveat_sentences_array)] = caveat['id']
                            caveat_sentences_array.append({
                                'text': misc_obj, 
                                 'id': len(caveat_sentences_array),
                                'type': 'misc@' + misc['name']})
                        else: # parameters and exception objects
                            for misc_sentence in misc_obj['sentences']:
                                if "the amount to add, not null" in misc_sentence:
                                    print(caveat)
                                sentence_id_to_caveat_id[len(caveat_sentences_array)] = caveat['id']
                                caveat_sentences_array.append({
                                    'text': misc_sentence, 
                                     'id': len(caveat_sentences_array),
                                    'type': 'misc@' + misc['name']})

{'name': 'plus', 'type': 'method', 'signature': 'default ChronoLocalDateTime<D> plus(TemporalAmount amount)', 'deprecated': False, 'sentences': ['Some example code indicating how and why this method is used: date = date.plus(period); // add a Period instance date = date.plus(duration); // add a Duration instance date = date.plus(workingDays(6)); // example user-written workingDays method Note that calling plus followed by minus is not guaranteed to return the same date-time.'], 'caveat_misc': [{'name': 'Parameters:', 'list': [{'parameter': 'amount', 'sentences': ['the amount to add, not null']}]}, {'name': 'Returns:', 'list': ['an object of the same type with the specified adjustment made, not null']}, {'name': 'Throws:', 'list': [{'exception': 'DateTimeException', 'sentences': ['if the addition cannot be made']}, {'exception': 'ArithmeticException', 'sentences': ['if numeric overflow occurs']}]}], 'id': 10152}
{'name': 'plus', 'type': 'method', 'signature': 'ChronoLocalDateTime<D> plu

In [49]:
for key in class_to_caveat_ids:
    if 29842 in class_to_caveat_ids[key]:
        print(key)

YearMonth
